In [8]:
import os
import json

def search_files_for_term(directory, search_term):
    """
    Searches for files in the given directory that contain the specified search_term.
    
    Parameters:
    - directory (str): The directory to search in.
    - search_term (str): The term to search for within the files.
    """
    # Check if the provided directory exists
    if not os.path.isdir(directory):
        print("The specified directory does not exist. Please check the path.")
        return

    # List to hold names of files containing the search term
    files_with_term = []
    files_with_term_path = []

    # Walking through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Construct the full file path
            file_path = os.path.join(root, file)
            try:
                # Open and read the file contents
                with open(file_path, 'r') as f:
                    contents_str = f.read()
                    if search_term in contents_str:
                        contents_dict = json.loads(contents_str)
                        files_with_term.append(contents_dict['result'])
                        files_with_term_path.append(file_path[len(directory):])
            except Exception as e:
                print(f"An error occurred while reading {file}: {e}")
    return files_with_term, files_with_term_path

In [9]:
# Call the function with the specified directory, search term
# Store into variable 'items'
items, items_paths = search_files_for_term(directory='../recipes',
                                          search_term='snc_item')
print(items_paths[0])

\drink\apple_juice.json


### Export into mcfunctions

In [10]:
def json_to_mcfunction(json_data, function_type):
    # Start building the command with the ID and basic structure
    command = f"{function_type} {json_data['id']}"

    # Add components
    if 'components' in json_data:
        components_strings = []
        for component_key, component_value in json_data['components'].items():
            # Special handling for nested 'custom_data'
            if component_key == 'custom_data':
                custom_data_strs = [f"{k}:{v}" for k, v in component_value.items()]
                components_strings.append(f"{component_key}={{{', '.join(custom_data_strs)}}}")
            else:
                # Directly turning the component's value into a string
                # {'minecraft:custom_name': "value"} becomes "minecraft:custom_name='value'"
                # Note: For non-string types, we don't use quotes.
                component_value_str = f"'{component_value}'" if isinstance(component_value, str) else str(component_value)
                components_strings.append(f"{component_key}={component_value_str}")

        # Combine components into the command
        components_str = ','.join(components_strings)
        command += f"[{components_str}]"

    # Add count at the end
    if 'count' in json_data:
        command += f" {json_data['count']}"

    return command
    

import shutil

def export_to_mcfunction(export_folder, function_type):
    # Delete the previous folder
    try:
        shutil.rmtree(export_folder)
        print("Folder \""+ export_folder +"\" deleted successfully! 🗑️✨")
    except OSError as e:
        print("Folder \""+ export_folder +"\" not found!")

    # Export all recipes into the 'export_folder'
    for i, item_path in enumerate(items_paths):
        # Split the path into the directory and the file name
        directory = os.path.dirname(export_folder+item_path)

        # Check if the directory exists, if not, create it
        if not os.path.exists(directory):
            os.makedirs(directory)  # This creates the directory and any intermediate directories

        # Now, safely open and write to your file
        with open(export_folder+item_path[:-5]+'.mcfunction', 'w') as file:
            file.write(json_to_mcfunction(items[i], function_type))
    print("Exported \""+ export_folder + "\" to .mcfunction\n")

# Give
export_to_mcfunction('../functions/api/give', 'give @s')
# Clear
# export_to_mcfunction('../functions/api/clear', 'clear @s')
# Item replace
export_to_mcfunction('../functions/api/item_replace', '$item replace entity @s $(slot) with')

Folder "../functions/api/give" deleted successfully! 🗑️✨
Exported "../functions/api/give" to .mcfunction

Folder "../functions/api/item_replace" deleted successfully! 🗑️✨
Exported "../functions/api/item_replace" to .mcfunction



In [11]:
def json_to_trades(json_data, count):
    # Start building the command with the ID and basic structure
    command = f"{{id:\"{json_data['id']}\", count:{count}"

    # Check for and add components if present
    if 'components' in json_data:
        # Initialize components part of the command
        command += ", components:{"

        components_strings = []
        for component_key, component_value in json_data['components'].items():
            # Special handling for nested 'custom_data'
            if component_key == 'custom_data':
                custom_data_strs = [f"{k}:{v}" for k, v in component_value.items()]
                components_strings.append(f"\"{component_key}\":{{{', '.join(custom_data_strs)}}}")
            else:
                # Format the component value as a string if it's of string type, otherwise, use its raw value
                component_value_str = f"'{component_value}'" if isinstance(component_value, str) else str(component_value)
                components_strings.append(f"\"{component_key}\":{component_value_str}")

        # Combine components into the command and close the components and overall command structures
        components_str = ','.join(components_strings)
        command += f"{components_str}}}"

    # Close the overall command structure if no components
    command += "}"

    return command

def export_to_trades(file_name, trades):
    command = "data merge entity @s {Offers:{Recipes:["
    for i, trade in enumerate(trades):
        with open(trade[0], 'r') as f:
            item = json.loads(f.read())
            command += "{buy:"+json_to_trades(item['result'], trade[1])
        with open(trade[2], 'r') as f:
            item = json.loads(f.read())
            command += ",sell:"+json_to_trades(item['result'], trade[3])+"}"
        # Add comma if is not the last element
        if i < len(trades) - 1:
            command += ","
        else:
            command += "]}}"
    # Now, safely open and write to your file
    with open('../functions/api/trade/'+file_name+'.mcfunction', 'w') as file:
        file.write(command)
        print(f"Trade \"{file_name}\" created successfully! ✨")

## Delete previous
try:
    shutil.rmtree("../functions/api/trade")
    os.makedirs("../functions/api/trade")
    print("Folder \"../functions/api/trade\" deleted successfully! 🗑️✨")
except OSError as e:
    print("Folder \"../functions/api/trade\" not found!")

villagers_path = "../recipes/villagers/"
survey_path = "../recipes/survey_corps/"
misc_path = "../recipes/misc/"
shifters_path = "../recipes/shifters/"

export_to_trades("survey",[
    [villagers_path+"golden_mark.json",2,survey_path+"uniform/hood.json",1],
    [villagers_path+"golden_mark.json",2,survey_path+"uniform/cape.json",1],
    [villagers_path+"golden_mark.json",2,survey_path+"uniform/harness.json",1],
    [villagers_path+"golden_mark.json",2,survey_path+"uniform/boots.json",1],
    [villagers_path+"golden_mark.json",1,misc_path+"gunpowder.json",3],
    [villagers_path+"golden_mark.json",2,misc_path+"jar.json",1],
    [villagers_path+"golden_mark.json",3,shifters_path+"syringe/empty.json",1],
    [villagers_path+"golden_mark.json",5,shifters_path+"syringe/full.json",1],
    [villagers_path+"golden_mark.json",16,shifters_path+"syringe/royal.json",1],
    [villagers_path+"golden_mark.json",32,villagers_path+"magus.json",1]
])

export_to_trades("guide",[
    [villagers_path+"emerald.json",1,misc_path+"guide/shingeki.json",1],
    [villagers_path+"emerald.json",3,misc_path+"guide/village.json",1],
    [villagers_path+"emerald.json",3,misc_path+"guide/love.json",1],
    [villagers_path+"emerald.json",5,misc_path+"guide/titan_war.json",1],
    [villagers_path+"emerald.json",8,misc_path+"guide/shipload.json",1],
    [villagers_path+"emerald.json",16,misc_path+"guide/shifters.json",1]
])

export_to_trades("toy",[
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/zick.json",1],
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/zwei.json",1],
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/michelle.json",1],
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/valentina.json",1],
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/vladlen.json",1],
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/allan.json",1],
    [villagers_path+"golden_mark.json",8,villagers_path+"toy/ameer.json",1],
    [villagers_path+"golden_mark.json",16,villagers_path+"toy/creepy.json",1],
    [villagers_path+"golden_mark.json",64,villagers_path+"toy/blocky.json",1],
    [villagers_path+"golden_mark.json",32,villagers_path+"toy/colossal.json",1]
])

export_to_trades("mask",[
    [villagers_path+"golden_mark.json",32,villagers_path+"mask/attack.json",1],
    [villagers_path+"golden_mark.json",32,villagers_path+"mask/cart.json",1],
    [villagers_path+"golden_mark.json",48,villagers_path+"mask/beast.json",1],
    [villagers_path+"golden_mark.json",64,villagers_path+"mask/colossal.json",1]
])

Folder "../functions/api/trade" deleted successfully! 🗑️✨
Trade "survey" created successfully! ✨
Trade "guide" created successfully! ✨
Trade "toy" created successfully! ✨
Trade "mask" created successfully! ✨


In [12]:
def json_to_slot(json_data, slot):
    command = f"Slot:{slot}b"

    for key, value in json_data.items():
        if 'components' in key:
            command += f",components:"
            components_strings = []
            for component_key, component_value in json_data['components'].items():
                # Special handling for nested 'custom_data'
                if component_key == 'custom_data':
                    custom_data_strs = [f"{k}:{v}" for k, v in component_value.items()]
                    components_strings.append(f"\"{component_key}\":{{{', '.join(custom_data_strs)}}}")
                else:
                    # Directly turning the component's value into a string
                    # {'minecraft:custom_name': "value"} becomes "minecraft:custom_name='value'"
                    # Note: For non-string types, we don't use quotes.
                    component_value_str = f"'{component_value}'" if isinstance(component_value, str) else str(component_value)
                    components_strings.append(f"\"{component_key}\":{component_value_str}")

            # Combine components into the command
            components_str = ','.join(components_strings)
            command += f"{{{components_str}}}"
        else:
            # General handling for other keys
            if isinstance(value, int):
                # Append 'b' to integers to match the target format's requirement
                command += f',{key}:{value}b'
            else:
                # Quotes around string values if not already present
                command += f',{key}:\"{value}\"'

    return command

In [13]:
set_chest = "setblock ~ ~ ~ chest{Items:["
chest = 0
for slot, item in enumerate(items):
    set_chest += "{"+json_to_slot(item, slot if slot <= 26 else slot-27)+"}"
    #if slot != len(items)-1:
    #    set_chest += ","
    if slot != 0 and slot % 26 == 0:
        chest += 1
        set_chest += "]}\nsetblock ~ ~"+str(chest)+" ~ chest{Items:["
    else:
        set_chest += ","
set_chest += "]}"

In [14]:
# Save into a new file
with open('../functions/api/all_items.mcfunction', 'w') as file:
    # Write the string to the file
    file.write(set_chest)
    print("All Items exported successfully! ✨")

All Items exported successfully! ✨
